In [ ]:
import boto3

def get_files_with_non_jpeg_content_type(bucket_name, s3_folder, region_name):
    """Lấy danh sách file có Content-Type khác 'image/jpeg' trong thư mục S3."""
    s3 = boto3.client('s3', region_name=region_name)
    files_with_non_jpeg = []

    # Liệt kê các file trong folder trên S3
    continuation_token = None
    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)

        if 'Contents' in response:
            for obj in response['Contents']:
                file_key = obj['Key']
                try:
                    # Kiểm tra metadata của file
                    file_metadata = s3.head_object(Bucket=bucket_name, Key=file_key)
                    content_type = file_metadata.get('ContentType', '')

                    # Nếu Content-Type khác 'image/jpeg', thêm vào danh sách
                    if content_type != 'image/jpeg':
                        files_with_non_jpeg.append(file_key)
                except Exception as e:
                    print(f"Không thể lấy metadata của file {file_key}: {e}")

        # Kiểm tra nếu có thêm dữ liệu
        if response.get('IsTruncated'):
            continuation_token = response.get('NextContinuationToken')
        else:
            break

    return files_with_non_jpeg


# Ví dụ gọi hàm
bucket_name = "hs-search-img-pdf"  # Thay bằng tên bucket của bạn
s3_folder = "Eximhow_img/"         # Thư mục trên S3
region_name = "ap-southeast-1"     # Region của bucket

# Gọi hàm để lấy danh sách file có Content-Type khác 'image/jpeg'
files_with_non_jpeg = get_files_with_non_jpeg_content_type(bucket_name, s3_folder, region_name)

# In ra kết quả
if files_with_non_jpeg:
    print("Danh sách các file có Content-Type khác 'image/jpeg':")
    for file in files_with_non_jpeg:
        print(file)
else:
    print("Không có file nào có Content-Type khác 'image/jpeg'.")


In [ ]:
import boto3

def update_content_type(bucket_name, s3_folder, files_to_upload, region_name):
    """Cập nhật Content-Type cho tất cả file trong list files_to_upload."""
    try:
        # Khởi tạo client S3
        s3 = boto3.client('s3', region_name=region_name)

        for file in files_to_upload:
            s3_file_path = f"{s3_folder}{file}"

            # Sửa Content-Type thành image/jpeg
            s3.copy_object(
                Bucket=bucket_name,
                CopySource={'Bucket': bucket_name, 'Key': s3_file_path},
                Key=s3_file_path,a
                Metadata={"Content-Type": "image/jpeg"},
                MetadataDirective="REPLACE"
            )
            print(f"Updated Content-Type for: s3://{bucket_name}/{s3_file_path}")

    except Exception as e:
        print(f"Lỗi khi cập nhật Content-Type: {e}")

# Sau khi tải lên file
update_content_type(bucket_name, s3_folder, files_with_non_jpeg, region_name)
